In [1]:
import numpy as np
import random
import math

In [ ]:
# def standard_regret(s, x, y):
#     return 0.5 * (s.win_matrix[x][y] + s.win_matrix[y][x])

In [24]:
class DuelingBandits:
    def __init__(self, num_arms, win_matrix, best_arm, time_horizon=1e10):
        self.best_arm = best_arm
        self.total_pulls = 0
        self.history = []
        self.time_horizon = time_horizon
        self.regret = 0
        self.num_arms = num_arms
        self.win_matrix = win_matrix  # Win matrix. win_matrix[i][j] is the probability that arm i beats arm j
        self.arms = {i:{'history':[], 'num_pulls':0, 'num_wins':0} for i in range(num_arms)}  # Set of arms
    
    def pull_arms(self, arm_1, arm_2):
        """Simulate pulling the selected arm and receiving a reward."""
        # Updates Globals:
        self.total_pulls += 1

        # Simulate pulling the arms
        if (np.random.binomial(1, self.win_matrix[arm_1, arm_2]+0.5)):
            winning_arm = arm_1
            losing_arm = arm_2
        else: 
            winning_arm = arm_2
            losing_arm = arm_1
        
        # Updates the number of pulls for each arm
        self.arms[arm_1]['num_pulls'] += 1
        self.arms[arm_2]['num_pulls'] += 1

        # Updates the number of wins for each arm
        self.arms[winning_arm]['num_wins'] += 1
        self.arms[losing_arm]['num_wins'] += 0

        # Add to the history of each of the arms:
        self.arms[arm_1]['history'].append({"winner": winning_arm, "loser": losing_arm, "round": self.total_pulls})
        self.arms[arm_2]['history'].append({"winner": winning_arm, "loser": losing_arm, "round": self.total_pulls})
        
        # Update the regret
        self.regret += 0.5 * (self.win_matrix[self.best_arm, losing_arm] + self.win_matrix[self.best_arm, winning_arm])

        # Add to the global history
        self.history.append({"winner": winning_arm, "loser": losing_arm, "round": self.total_pulls, "regret": self.regret})
        return winning_arm
    

In [3]:
# e_i,j = 0.1 for all b_i > b_j (beat the mean example 1)
K = 5 # number of bandits
T = 1e10 # time horizon

Eps = np.zeros((K, K))
for i in range(K):
    for j in range(i+1, K):
        Eps[i, j] = -0.1
        Eps[j, i] = 0.1

In [25]:
bandit = DuelingBandits(K, Eps, K-1, T)

In [26]:
def round(b_1, b_2, eps):
    # returns the result for draw b_1 vs. b_2
    return np.random.binomial(1, eps[b_1, b_2]+0.5)

In [27]:
res = np.empty((1000, 1))
for i in range(1000):
    res[i] = round(1, 1, Eps)

print(sum(res))

[504.]


In [28]:
def IF1(T, K, bandit):
    # runs interleaved filter 1 over K dueling-bandits with time horizon T
    wins = np.zeros((K, K))
    P_hat = np.ones((K, K))*1/2
    n_rounds = np.zeros((K, K))
    c_hat = np.empty((K, K))
    delta = 1/(T*K**2)
    T_hat = 0 # total comparisons made
    def update(b_1, b_2, res):
        n_rounds[b_1, b_2] = n_rounds[b_1, b_2] + 1
        n_rounds[b_2, b_1] = n_rounds[b_2, b_1] + 1
        wins[b_1, b_2] += res
        wins[b_2, b_1] += (1-res)
        P_hat[b_1, b_2] = wins[b_1, b_2]/n_rounds[b_1, b_2]
        P_hat[b_2, b_1] = wins[b_2, b_1]/n_rounds[b_2, b_1]
        c_hat[b_1, b_2] = math.sqrt(math.log(1/delta)/n_rounds[b_1, b_2])
        c_hat[b_2, b_1] = math.sqrt(math.log(1/delta)/n_rounds[b_2, b_1])

    W = np.arange(0, K)
    b_hat = np.random.choice(K)
    W = np.array(W[~np.isin(W, b_hat)])
    while len(W) > 0:
        for b in W:
            res = bandit.pull_arms(b_hat, b)
            if res == b_hat:
                won = 1
            else:
                won = 0
            T_hat += 1
            update(b_hat, b, won)
        b_to_remove = np.where(np.logical_and(P_hat[b_hat, :] > 1/2, 
            (P_hat[b_hat, :] - c_hat[b_hat, :]) > 1/2))
        W = np.array(W[~np.isin(W, b_to_remove)])
        b_win = np.where(np.logical_and(P_hat[b_hat, :] < 1/2, 
            (P_hat[b_hat, :] + c_hat[b_hat, :]) < 1/2))[0]
        if len(b_win) > 0:
            b_hat = b_win[np.random.choice(len(b_win))]
            W = np.array(W[(~(np.isin(W, b_hat)))])
            wins = np.zeros((K, K))
            P_hat = np.ones((K, K))*1/2
            n_rounds = np.zeros((K, K))
            c_hat = np.empty((K, K))
    
    return (b_hat, T_hat)

In [29]:
bandit = DuelingBandits(K, Eps, K-1, T)
print(IF1(T, K, bandit))

[0 1 2 3]
(4, 4)


In [ ]:
def IF2(T, K, bandit):
    # runs interleaved filter 2 over K dueling-bandits with time horizon T
    wins = np.zeros((K, K))
    P_hat = np.ones((K, K))*1/2
    n_rounds = np.zeros((K, K))
    c_hat = np.empty((K, K))
    delta = 1/(T*K**2)
    T_hat = 0 # total comparisons made
    
    def update(b_1, b_2, res):
        #res = 1 if b_1 beat b_2, 0 otherwise
        n_rounds[b_1, b_2] = n_rounds[b_1, b_2] + 1
        n_rounds[b_2, b_1] = n_rounds[b_2, b_1] + 1
        wins[b_1, b_2] += res
        wins[b_2, b_1] += (1-res)
        P_hat[b_1, b_2] = wins[b_1, b_2]/n_rounds[b_1, b_2]
        P_hat[b_2, b_1] = wins[b_2, b_1]/n_rounds[b_2, b_1]
        c_hat[b_1, b_2] = math.sqrt(math.log(1/delta)/n_rounds[b_1, b_2])
        c_hat[b_2, b_1] = math.sqrt(math.log(1/delta)/n_rounds[b_2, b_1])

    W = np.arange(0, K)
    b_hat = np.random.choice(K)
    W = np.array(W[~np.isin(W, b_hat)])
    while len(W) > 0:
        for b in W:
            res = bandit.pull_arms(b_hat, b)
            if res == b_hat:
                won = 1
            else:
                won = 0
            T_hat += 1
            update(b_hat, b, won)
        b_to_remove = np.where(np.logical_and(P_hat[b_hat, :] > 1/2, 
            (P_hat[b_hat, :] - c_hat[b_hat, :]) > 1/2))
        W = np.array(W[~np.isin(W, b_to_remove)])
        b_win = np.where(np.logical_and(P_hat[b_hat, :] < 1/2, 
            (P_hat[b_hat, :] + c_hat[b_hat, :]) < 1/2))[0]
        if len(b_win) > 0:
            # pruning step
            b_to_remove = np.where(P_hat[b_hat, :] > 1/2)
            W = np.array(W[(~(np.isin(W, b_to_remove)))])

            # choose new b_hat from remaining
            b_hat = b_win[np.random.choice(len(b_win))]
            W = np.array(W[(~(np.isin(W, b_hat)))])
            wins = np.zeros((K, K))
            P_hat = np.ones((K, K))*1/2
            n_rounds = np.zeros((K, K))
            c_hat = np.empty((K, K))
    
    return (b_hat, T_hat)

In [ ]:
bandit = DuelingBandits(K, Eps, K-1, T)
print(IF2(T, K, bandit))

In [ ]:
def btmb(T, K, bandit):
    # runs beat the mean bandit for K dueling-bandits with time horizon T
    delta = 1/(2*K*T)
    
    def confidence(n)
        # returns confidence interval for this many draws
        return math.sqrt(1/n * math.log(1/delta))
    
    W = np.arange(0, K)
    num_comp = np.zeros(K)
    num_wins = np.zeros(K)
    P_hat = np.ones(K)*1/2
    # choose random bandit to start with out of those with fewest pulls
    n_star = 0
    c_star = 1
    t = 0
    while len(W) > 1 and t < T
        b = np.random.choice(np.flatnonzero(num_comp == np.min(num_comp)))
        b_prime = np.random.choice(W)
        res = bandit.pull_arms(b, b_prime)
        if b == res:
            num_wins(b) += 1
            P_hat(b) = num_wins(b)/num_comp(b)
            
        num_comp(b) += 1
        c_star = confidence(num_comp)
        t += 1
        if np.min(P_hat + c_star) <= np.max(P_hat - c_star):
            b_prime = np.random.choice(np.flatnonzero(P_hat == np.min(P_hat)))
            
            
            
            


In [25]:
class sbm():
	def __init__(self, n_arms, alpha=1):
		self.alpha = alpha
		self.num_arms = n_arms

	def reset(self):
		self.averages = np.ones(self.num_arms)*math.inf
		self.pulls = np.ones(self.num_arms)
		self.time = 1

	def advance(self):
		bound = self.averages + np.sqrt((self.alpha+2)*np.log(self.time)/(2*self.pulls))
		choice = np.argmax(bound)
		return choice

	def feedback(self, arm, reward):
		self.time += 1
		
		if self.averages[arm] == math.inf:
			self.averages[arm] = reward
		else:
			self.averages[arm] = (self.averages[arm]*self.pulls[arm] + reward)/(self.pulls[arm]+1)
		
		self.pulls[arm] += 1

In [13]:
class Doubler():
	def __init__(self, bandit : DuelingBandits, regret_func):

		self.bandit = bandit
		self.num_arms = bandit.num_arms
		self.sbm = sbm(bandit.num_arms)

		self.l = np.random.randint(self.num_arms, size=1)
		self.i, self.t = 1, 1

		self.time_horizon = bandit.time_horizon
		self.regret_func = regret_func

	def run(self):

		regret = [0]
		
		while self.t < self.time_horizon:
						
			self.sbm.reset()
			new_l = set()

			for j in range(2**self.i):
				xt = np.random.choice(self.l)
				yt = self.sbm.advance()
				new_l.add(yt)
				winning_arm = self.bandit.pull_arms(xt, yt)

				if xt == winning_arm:
					self.sbm.feedback(yt, 0)
					self.sbm.feedback(xt, 1)
				else:
					self.sbm.feedback(yt, 1)
					self.sbm.feedback(xt, 0)

				regret.append(regret[-1]+self.regret_func(xt,yt))

				self.t += 1

				if self.t >= self.time_horizon:
					break

			self.l = np.array(list(new_l))
			self.i += 1

		return list(np.around(regret,3)), np.argmax(self.sbm.averages)

In [43]:
class MultiSBM():
	def __init__(self, bandit : DuelingBandits, regret_func):

		self.bandit = bandit
		self.num_arms = bandit.num_arms
		self.sbms = [sbm(bandit.num_arms) for i in range(self.num_arms)]

		self.t = 1

		self.time_horizon = bandit.time_horizon
		self.regret_func = regret_func

	def run(self):
		for cursbm in self.sbms:
			cursbm.reset()
		yt_1 = np.random.choice(range(1, self.num_arms))
		while self.t < self.time_horizon:
	
			xt = yt_1
			cur_sbm = self.sbms[xt]
			yt = cur_sbm.advance()

			
			winning_arm = self.bandit.pull_arms(xt, yt)

			if xt == winning_arm:
				cur_sbm.feedback(yt, 0)
				cur_sbm.feedback(xt, 1)
			else:
				cur_sbm.feedback(yt, 1)
				cur_sbm.feedback(xt, 0)
				
				# Handling Regret here
				#self.regret.append(self.regret[-1]+self.regret_func(xt,yt))

				self.t += 1

				if self.t >= self.time_horizon:
					break
		return #list(np.around(regret, 3)), np.argmax(self.sbm.averages)

In [34]:
# Create the following win matrix:
#i,j = 1/(1 + exp(µj − µi)) − 0.5 for all i 6= j (beat the mean example 2)
K = 5 # number of bandits
T = 1e10 # time horizon
# Construct 5 examples of normals with different means
mu = np.random.normal(size=K)
print(mu)
preferences = np.zeros((K, K))
for i in range(K):
    for j in range(i+1, K):
        preferences[i, j] = 1/(1 + math.exp(mu[j] - mu[i])) - 0.5
        preferences[j, i] = 1/(1 + math.exp(mu[i] - mu[j])) - 0.5
preferences


[ 0.81848889 -1.12970365 -0.32405647 -0.3819822   1.63734184]


array([[ 0.        ,  0.37524942,  0.25814666,  0.26860858, -0.1939928 ],
       [-0.37524942,  0.        , -0.19118117, -0.17868201, -0.44086883],
       [-0.25814666,  0.19118117,  0.        ,  0.01447738, -0.3766842 ],
       [-0.26860858,  0.17868201, -0.01447738,  0.        , -0.3828111 ],
       [ 0.1939928 ,  0.44086883,  0.3766842 ,  0.3828111 ,  0.        ]])

In [44]:
v = MultiSBM(DuelingBandits(5, preferences, 4, 1e5), lambda x,y: 0.5*(preferences[x,y]+preferences[y,x]))
v.run()

/var/folders/_r/krsqqcnn117gmqmty9_pdlfc0000gn/T/ipykernel_17459/74170729.py:7: RuntimeWarning: invalid value encountered in multiply
  self.averages = np.ones(self.num_arms)*math.inf


In [45]:
list(map(lambda x: x['regret'], v.bandit.history))

[0.288401946914699,
 0.7002419081403215,
 1.0799895566354003,
 1.2713951042039568,
 1.5597970511186559,
 1.9716370123442784,
 2.163042559912835,
 2.451444506827534,
 2.642850054396091,
 2.8342556019646477,
 3.0256611495332044,
 3.437501110758827,
 3.628906658327384,
 3.917308605242083,
 4.10871415281064,
 4.300119700379196,
 4.491525247947752,
 4.903365209173375,
 5.283112857668454,
 5.571514804583153,
 5.762920352151709,
 6.051322299066408,
 6.242727846634964,
 6.531129793549663,
 6.72253534111822,
 7.010937288032919,
 7.202342835601475,
 7.490744782516174,
 7.68215033008473,
 7.970552276999429,
 8.161957824567986,
 8.450359771482685,
 8.738761718397384,
 9.150601679623007,
 9.342007227191564,
 9.533412774760121,
 9.724818322328678,
 9.916223869897236,
 10.204625816811934,
 10.584373465307012,
 10.775779012875569,
 11.064180959790267,
 11.255586507358824,
 11.543988454273524,
 11.735394001842081,
 12.023795948756781,
 12.215201496325339,
 12.406607043893896,
 12.598012591462453,
 12.7

In [48]:
class Sparring():
	def __init__(self, bandit : DuelingBandits, regret_func):

		self.bandit = bandit
		self.num_arms = bandit.num_arms
		self.sl = sbm(bandit.num_arms)
		self.sr = sbm(bandit.num_arms)

		self.t = 1

		self.time_horizon = bandit.time_horizon
		self.regret_func = regret_func

	def run(self):
		# Resetting the SBMs
		self.sl.reset()
		self.sr.reset()
		
		while self.t < self.time_horizon:
	
			xt = self.sl.advance()
			yt = self.sr.advance()
			
			winning_arm = self.bandit.pull_arms(xt, yt)

			if xt == winning_arm:
				self.sl.feedback(yt, 0)
				self.sl.feedback(xt, 1)
				self.sr.feedback(yt, 1)
				self.sr.feedback(xt, 0)
			else:
				self.sl.feedback(yt, 1)
				self.sl.feedback(xt, 0)
				self.sr.feedback(yt, 0)
				self.sr.feedback(xt, 1)
				
				# Handling Regret here
				#self.regret.append(self.regret[-1]+self.regret_func(xt,yt))

			self.t += 1

			if self.t >= self.time_horizon:
				break
		return #list(np.around(regret, 3)), np.argmax(self.sbm.averages)


In [49]:
v = Sparring(DuelingBandits(5, preferences, 4, 1e5), lambda x,y: 0.5*(preferences[x,y]+preferences[y,x]))
v.run()

/var/folders/_r/krsqqcnn117gmqmty9_pdlfc0000gn/T/ipykernel_17459/74170729.py:7: RuntimeWarning: invalid value encountered in multiply
  self.averages = np.ones(self.num_arms)*math.inf


In [50]:
list(map(lambda x: x['regret'], v.bandit.history))

[0.1939927986922848,
 0.6348616260064166,
 1.0115458278594613,
 1.3943569229965744,
 1.3943569229965744,
 1.5883497216888594,
 2.0292185490029913,
 2.4059027508560358,
 2.788713845993149,
 2.788713845993149,
 2.982706644685434,
 3.4235754719995657,
 3.80025967385261,
 4.183070768989723,
 4.183070768989723,
 4.377063567682008,
 4.81793239499614,
 5.194616596849184,
 5.577427691986298,
 5.577427691986298,
 5.771420490678582,
 6.212289317992714,
 6.588973519845759,
 6.971784614982872,
 6.971784614982872,
 7.165777413675157,
 7.6066462409892885,
 7.983330442842333,
 8.366141537979447,
 8.366141537979447,
 8.560134336671732,
 9.001003163985864,
 9.37768736583891,
 9.760498460976022,
 9.760498460976022,
 9.954491259668307,
 10.39536008698244,
 10.772044288835485,
 11.154855383972597,
 11.154855383972597,
 11.348848182664883,
 11.789717009979015,
 12.16640121183206,
 12.549212306969173,
 12.549212306969173,
 12.743205105661458,
 13.18407393297559,
 13.560758134828635,
 13.943569229965748,
 13